In [7]:
require 'poisson'

class Modeling
  def initialize(people_num, possibility_correct)
    @people_num = people_num
    @people_half = half_num(@people_num) # 終了の値や半分の値に用いる
    @possibility_correct = possibility_correct
  end

  # 一人目の意見を採用するという雑魚アルゴリズム
  # TODO 時刻0から99で行なっているが1から100の方が好ましい気がする
  def baseline_method_deciding_by_first_person_with_poisson(possibility_correct)
    collecting_deadline = 60 # 意見収集の締め切り 59時刻
    poisson = Poisson.new(30) # 平均30人のポアソン分布
    average_method_utility = 0
    # 0~59人来るときのみを考慮する(それ以外は0に近似する)
    60.times do |t_people|
      # t_people人来るとき
      when_people_come = Array.new(collecting_deadline) # 人がいつ来るか配列で表す
      temp_probability_by_poisson = poisson.probability{ |x| x == t_people }
      t_people.times do |t|
        when_people_come[t] = 1
      end
      method_utility = 0 # 効用 = 精度(正解率) - かかる時間(決定する時刻 / collecting_deadline)
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ないときは飛ばす
        weight = 0.5.to_f / collecting_deadline # 時間がかかることに対する重み
        method_utility += possibility_correct - index.to_f * weight
        break
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 5人で多数決して意見集約を行うというアルゴリズム
  def baseline_method_deciding_by_majority_vote_with_poisson(possibility_correct)
    collecting_deadline = 60 # 意見収集の締め切り 59時刻
    poisson = Poisson.new(30) # 平均30人のポアソン分布
    average_method_utility = 0
    # 0~59人来るときのみを考慮する(それ以外は0に近似する)
    60.times do |t_people|
      # t_people人来るとき
      when_people_come = Array.new(collecting_deadline) # 人がいつ来るか配列で表す
      temp_probability_by_poisson = poisson.probability{ |x| x == t_people }
      t_people.times do |t|
        when_people_come[t] = 1
      end
      method_utility = 0 # 効用 = 精度(正解率) - かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      next if t_people < 5 # 5人集まらない場合は、utility = 0
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count += 1
        if people_count == 5
          weight = 0.5.to_f / collecting_deadline
          method_utility +=  (1 - relative_error_by_majority_vote(people_count, possibility_correct)) - index.to_f * weight
          break
        end
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 先に3人集まった方の意見を採用するアルゴリズム
  def baseline_method_deciding_by_half_opinion_with_poisson(possibility_correct)
    collecting_deadline = 60 # 意見収集の締め切り 59時刻
    poisson = Poisson.new(30) # 平均30人のポアソン分布
    average_method_utility = 0
    # 0~59人来るときのみを考慮する(それ以外は0に近似する)
    60.times do |t_people|
      # t_people人来るとき
      when_people_come = Array.new(collecting_deadline) # 人がいつ来るか配列で表す
      temp_probability_by_poisson = poisson.probability{ |x| x == t_people }
      t_people.times do |t|
        when_people_come[t] = 1
      end
      method_utility = 0 # 効用 = 精度(正解率) - かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      people_count_arr = [] # EXAMPLE [4, 9, 20, 40, 45] 一人目は時刻4にきた　二人目は時刻9にきた...
      next if t_people < 5 # 5人集まらない場合は、utility = 0
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count_arr << index
        people_count += 1
        if people_count == 5
          weight = 0.5.to_f / collecting_deadline
          relative_error_array = relative_error_array_by_half_opinion(half_num(people_count), possibility_correct)
          expected_error = relative_error_array.inject(:+)
          average_index = 0
          relative_error_array.each_with_index do |e, i| # もっと簡単にかけそう
            average_index += people_count_arr[i + 2] * (e / expected_error)
          end
          method_utility =  (1 - expected_error) - average_index.to_f * weight
          break
        end
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 誰か一人集まりかつ、時刻9で意見収集を打ち切り,多数決で意思決定を行う
  def baseline_method_deciding_by_time_limit_with_poisson(possibility_correct)
    collecting_deadline = 60 # 意見収集の締め切り 59時刻
    poisson = Poisson.new(30) # 平均30人のポアソン分布
    average_method_utility = 0
    # 0~59人来るときのみを考慮する(それ以外は0に近似する)
    60.times do |t_people|
      # t_people人来るとき
      when_people_come = Array.new(collecting_deadline) # 人がいつ来るか配列で表す
      temp_probability_by_poisson = poisson.probability{ |x| x == t_people }
      t_people.times do |t|
        when_people_come[t] = 1
      end
      method_utility = 0 # 効用 = 精度(正解率) - かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      # people_count_arr = [] # EXAMPLE [4, 9, 20, 40, 45] 一人目は時刻4にきた　二人目は時刻9にきた...
      # next if t_people < 5 # 5人集まらない場合は、utility = 0
      when_people_come.shuffle.each_with_index do |elem, index|
        if index >= 9 && people_count >= 1
          weight = 0.5.to_f / collecting_deadline
          method_utility = (1 - relative_error_by_majority_vote(people_count, possibility_correct)) - index.to_f * weight
          break
        end
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count += 1
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  def half_num(num)
    (num.to_f / 2).ceil
  end

  # nCk 実行例 5C2 = 5 * 4 / 2! = 10
  def combi(n, k)
    k = n - k if 2 * k > n
    return 1 if k == 0
    ((n - k + 1)..n).reduce(&:*) / (1..k).reduce(&:*)
  end

  # 誤差率をもとめる.先にある割合の意見(finish_num)を集めた方を意見集約の結果とするアルゴリズム
  # このアルゴリズムは必要な人数が動的に変化するので、誤差率をfinish_num個の要素にした配列で返す
  def relative_error_array_by_half_opinion(finish_num, possibility_correct) # possibility_correct => 人が正解する確率p
    relative_error = []
    finish_num.times do |t|
      relative_error << ((1 - possibility_correct)**finish_num) * (possibility_correct**t) * combi(finish_num - 1 + t, t)
    end
    relative_error # EXAMPLE (finish_num = 3 )[0.2, 0.3, 0.5] 0.2は3:0で決まるとき 0.3は3:1で決まるとき 0.5は3:2で決まるとき
  end

  # 誤差率をもとめる.people_num人で多数決を行い意見集約の結果とするアルゴリズム
  def relative_error_by_majority_vote(people_num, possibility_correct) # possibility_correct => 人が正解する確率p
    half_num = half_num(people_num)
    relative_error = 0
    half_num.times do |t|
      relative_error += (possibility_correct**t) * ((1 - possibility_correct)**(people_num - t)) * combi(people_num, t)
    end
    relative_error
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method1(possibility_correct) # possibility_correct => 人が正解する確率p
    # 終了する人数を求める
    finish_num = @people_half
    # 誤差率を求め、返す
    relative_error_array_by_half_opinion(finish_num, possibility_correct).inject(:+)
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method2(possibility_correct)
    # 多数決を行い誤差率を求め、返す
    relative_error_by_majority_vote(@people_num, possibility_correct)
  end

  # 誤差率εを変動させて、必要な人数Xを求める
  def baseline_method3(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 終了する人数を求める
    temp_finish_num = half_num(people_num)
    # 必要な平均人数
    average_num = 0
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      relative_error_array = relative_error_array_by_half_opinion(temp_finish_num, @possibility_correct)
      expected_error = relative_error_array.inject(:+)
      average_num = 0
      relative_error_array.each_with_index do |e, i| # もっと簡単にかけそう
        average_num += e * (i + temp_finish_num) / expected_error
      end
      break if expected_error <= relative_error
      people_num += 2
      temp_finish_num = half_num(people_num)
    end
    # 必要な人数の平均を返す
    average_num
  end

  # 多数決を意見集約のアルゴリズムとする
  def baseline_method4(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      expected_error = relative_error_by_majority_vote(people_num, @possibility_correct)
      break if expected_error <= relative_error
      people_num += 2
    end
    # 必要な人数を返す
    people_num
  end
end

require 'rbplotly'

PEOPELE_NUM = 199
POSSIBILITY_CORRECT = 0.7
DELIMITER = 100

model = Modeling.new(PEOPELE_NUM, POSSIBILITY_CORRECT)

# 縦軸 => 効用utility, 横軸 => 人の正解する確率p
utility_possibility_x_axis = (DELIMITER / 2..DELIMITER).to_a
# 乱数を用いてるので10回分の平均をとる
temp_utility_possibility_y_axis_by_first_person = []
temp_utility_possibility_y_axis_by_majority_vote = []
temp_utility_possibility_y_axis_by_time_limit = []
10.times do
  temp_utility_possibility_y_axis_by_first_person << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_first_person_with_poisson(e.to_f / DELIMITER) }
  temp_utility_possibility_y_axis_by_majority_vote << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_majority_vote_with_poisson(e.to_f / DELIMITER) }
  temp_utility_possibility_y_axis_by_time_limit << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_time_limit_with_poisson(e.to_f / DELIMITER) }
end
utility_possibility_y_axis_by_first_person = []
utility_possibility_y_axis_by_majority_vote = []
utility_possibility_y_axis_by_time_limit = []
utility_possibility_x_axis.length.times do |x_axis|
  average_y_first_person = 0
  average_y_majority_vote = 0
  average_y_time_limit = 0
  10.times do |t|
    average_y_first_person +=  temp_utility_possibility_y_axis_by_first_person[t][x_axis] / 10
    average_y_majority_vote +=  temp_utility_possibility_y_axis_by_majority_vote[t][x_axis] / 10
    average_y_time_limit +=  temp_utility_possibility_y_axis_by_time_limit[t][x_axis] / 10
  end
  utility_possibility_y_axis_by_first_person << average_y_first_person
  utility_possibility_y_axis_by_majority_vote << average_y_majority_vote
  utility_possibility_y_axis_by_time_limit << average_y_time_limit
end
utility_possibility_by_first_person_trace = [ {x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_first_person}]
utility_possibility_by_majority_vote_trace = [ {x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_majority_vote}]
utility_possibility_by_time_limit_trace = [ {x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_time_limit}]

pl = Plotly::Plot.new(data: utility_possibility_by_first_person_trace + utility_possibility_by_majority_vote_trace + utility_possibility_by_time_limit_trace)
pl.layout.xaxis = { title: 'possibility_correct' }
pl.layout.yaxis = { title: 'utility' }
pl.show

(pry):1787: warning: already initialized constant PEOPELE_NUM
(pry):1526: warning: previous definition of PEOPELE_NUM was here
(pry):1788: warning: already initialized constant POSSIBILITY_CORRECT
(pry):1527: warning: previous definition of POSSIBILITY_CORRECT was here
(pry):1789: warning: already initialized constant DELIMITER
(pry):1528: warning: previous definition of DELIMITER was here


#<Plotly::Offline::HTML:0x00007f8c7190e570 @id="1589f9f9-689b-4485-ac21-55b3d79d710c", @data=[{:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.49088220249123504, 0.5023435585195182, 0.5106021279893912, 0.5216537149294289, 0.5314243788162347, 0.5411435598409717, 0.5510711151509865, 0.5617261643594804, 0.5700100704543023, 0.580563708189395, 0.5926137371806354, 0.600593649414226, 0.6123637156695727, 0.6205640388340087, 0.6321824137346185, 0.641220955741954, 0.6508812336745463, 0.6599571823235403, 0.6729623517029888, 0.682896657421412, 0.6904583842457972, 0.6999306072758473, 0.7101769770925068, 0.7228934745712929, 0.7317147451394054, 0.7398835046158574, 0.751471754920062, 0.7626705427360886, 0.7710156798014466, 0.7816622431923875, 0.7911898724492868, 0.8010565539053911, 0.811933727613147, 0.8219739597038753, 0.8330275402507417, 0.8408619960129576, 0.8509725319661512, 0.8626811364323664, 0.8697533303341687, 0.8813764921551497, 0.8912729904102966, 0.8993878811653836, 0.911012595268595, 0.9214121869455897, 0.9313155088435833, 0.9407497565363302, 0.9517875956675276, 0.960753763854279, 0.9716022960816328, 0.9838151725655375, 0.9904255937808653]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.4215262927322001, 0.4426308073049783, 0.46179954655170924, 0.48230809694471977, 0.49981470814918166, 0.5153107839852528, 0.533963424493641, 0.5529966131436306, 0.5725139123445614, 0.5879423715044605, 0.6070232174460881, 0.6211699020877377, 0.6370218322675077, 0.6549542880624741, 0.6743677573234512, 0.6874419340197625, 0.7018067591357019, 0.7196113729667823, 0.7353353489085686, 0.7464173230519022, 0.7593086055928705, 0.7717523484695175, 0.7826367175014772, 0.8004449338719811, 0.8106529816042594, 0.820663021526809, 0.8272140335355587, 0.8403122412356819, 0.8491807710858188, 0.8578655637031454, 0.8653937964115137, 0.8725361008259995, 0.8804055410828883, 0.8872621899885236, 0.8922459918075442, 0.8955501862668134, 0.9004236095598308, 0.905349456059963, 0.9129878050346546, 0.9128108502467653, 0.9125443539655808, 0.9167978967208794, 0.919975400179418, 0.9210324165377409, 0.9229626234167568, 0.9233532197291606, 0.9211623735214391, 0.9204129168902515, 0.9224000102089693, 0.9235879443017295, 0.9230818771716931]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.5175427255915545, 0.5291082847181063, 0.5658084030762589, 0.5672873799402094, 0.5860981609780095, 0.5898578613912157, 0.6178750978892205, 0.6333680958112081, 0.6480729440997509, 0.6559290232477943, 0.6655202097424123, 0.6885718244462731, 0.6967707031880168, 0.7146407106199321, 0.7305247614075627, 0.7426976283262801, 0.7467320358152696, 0.761453230258689, 0.7770783478892207, 0.7855873554984723, 0.7960299337611454, 0.8055967134707178, 0.815130167468207, 0.8255489263572262, 0.8328754426894412, 0.8377449664282047, 0.8529354135745416, 0.855652381369424, 0.8582814050909071, 0.8662482158919692, 0.8759473655758061, 0.8770145360754736, 0.8876035798470367, 0.8881288412464211, 0.8964381494927329, 0.8985363974209426, 0.9034218884274525, 0.9038146690807509, 0.9076060536879247, 0.9102817076069849, 0.9147542663799492, 0.9161967828217044, 0.9170532110785491, 0.9188775390639874, 0.9213015446655661, 0.9218227486097759, 0.9228893189039128, 0.9234696166653034, 0.9240650996947991, 0.9247376250383648, 0.9249571304258927]}], @layout={:xaxis=>{:title=>"possibility_correct"}, :yaxis=>{:title=>"utility"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7f8c72f290d0 last_endpoint="tcp://127.0.0.1:62242">